# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 5

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        # input is 28x28x1
        # conv1(kernel=5, filters=10) 28x28x10 -> 24x24x10
        # max_pool(kernel=2) 24x24x10 -> 12x12x10
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        
        # conv2(kernel=5, filters=20) 12x12x10 -> 8x8x20
        # max_pool(kernel=2) 8x8x20 -> 4x4x20
        x = F.relu(F.max_pool2d(self.dropout(self.conv2(x)), 2))
        
        # flatten 4x4x20 = 320
        x = x.view(-1, 320)
        
        # 320 -> 50 -> 10
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        # transform to logits
        return F.log_softmax(x,dim=1)


model = SimpleNet()

criterion = F.nll_loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

Next, we can start to train and evaluate!

In [4]:
train_accuracy = 0
test_accuracy = 0

# train and evaluate
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    train_correct = 0
    for images, labels in tqdm(train_loader):
        data = Variable(images)
        target = Variable(labels)
        
        optimizer.zero_grad()
        preds = model(data)
        loss = criterion(preds, target)
        train_loss += loss.data.item()
        pred = preds.data.max(1)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data).cpu().sum()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(train_loader)
    train_accuracy = 100. * train_correct / len(train_loader.dataset)
    
    # evaluate
    model.eval() # set model in inference mode (need this because of dropout)
    test_loss = 0
    test_correct = 0
    for data, target in test_loader:
        with torch.no_grad():
            data = Variable(data) 
            target = Variable(target)
        
        preds = model(data)
        test_loss += criterion(preds, target).data.item()
        pred = preds.data.max(1)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data).cpu().sum()
    
    test_loss /= len(test_loader) # loss function already averages over batch size
    test_accuracy = 100. * test_correct / len(test_loader.dataset)
       
    print("\nEpoch %d" % epoch)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        train_accuracy))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        test_accuracy))

  0%|          | 2/468 [00:00<00:25, 18.00it/s]


Epoch 0
Train set: Average loss: 1.5995, Accuracy: 26749/60000 (44%)
Test set: Average loss: 0.4710, Accuracy: 8734/10000 (87%)



  0%|          | 2/468 [00:00<00:24, 19.33it/s]


Epoch 1
Train set: Average loss: 0.6339, Accuracy: 48110/60000 (80%)
Test set: Average loss: 0.2429, Accuracy: 9268/10000 (92%)



  0%|          | 2/468 [00:00<00:25, 18.28it/s]


Epoch 2
Train set: Average loss: 0.4596, Accuracy: 51673/60000 (86%)
Test set: Average loss: 0.1680, Accuracy: 9463/10000 (94%)



  0%|          | 2/468 [00:00<00:35, 13.12it/s]


Epoch 3
Train set: Average loss: 0.3755, Accuracy: 53168/60000 (88%)
Test set: Average loss: 0.1365, Accuracy: 9560/10000 (95%)



100%|██████████| 468/468 [00:23<00:00, 19.65it/s]



Epoch 4
Train set: Average loss: 0.3276, Accuracy: 54169/60000 (90%)
Test set: Average loss: 0.1143, Accuracy: 9632/10000 (96%)



#### Q5:
Please print the training and testing accuracy.

In [5]:
print('Training Accuracy: {:.0f}%'.format(train_accuracy))
print('Testing Accuracy: {:.0f}%'.format(test_accuracy))

Training Accuracy: 90%
Testing Accuracy: 96%
